# WeatherWise Advisor

#### Download necessary libraries

In [1]:
!pip install fetch-my-weather
!pip install hands-on-aib

ERROR: Could not find a version that satisfies the requirement hands-on-aib (from versions: none)
ERROR: No matching distribution found for hands-on-aib


#### Import Libraries

In [2]:
# Plotly configuration for different environments
# --- Global imports ---
import sys
import plotly.io as pio

# ✅ Auto-adjust Plotly renderer for environment compatibility (must run before all plotting code)
if "google.colab" in sys.modules:
    pio.renderers.default = "colab"
else:
    pio.renderers.default = "notebook_connected"   

# --- Import other required modules ---
import plotly.express as px

import os
import requests
import pandas as pd
import importlib
import time

# --- Graphing libraries ---
import matplotlib.pyplot as plt
import seaborn as sns

# --- Chatbot & NLP libraries ---
import re
import nltk

# --- UI design libraries ---
from ipywidgets import VBox, HBox, Dropdown, Button, Output
import pyinputplus as pyip
from IPython.display import display, clear_output
import ipywidgets as W
import ipywidgets as widgets
from IPython.display import display, HTML

# --- Weather data & utilities ---
from fetch_my_weather import get_weather
from datetime import datetime
from functools import lru_cache
from typing import List, Iterable

## 🌤️ Weather Data Functions

In [3]:
def get_weather_data(location, forecast_days=5):
    """
    Retrieve weather data for a specified location (1–5 day forecast).

    This function fetches current and forecast weather data using the 
    `get_weather()` function from the local weather-fetching module. 
    It processes and returns both the current conditions and a daily 
    forecast summary in a structured format.

    Args:
        location (str): City or location name.
        forecast_days (int): Number of days to forecast (1–5).

    Returns:
        dict: Weather data including:
            {
                "current_condition": {...},   # Current weather information
                "forecast": pandas.DataFrame  # Forecast data table
            }
    """

    # ✅ Retrieve raw data from custom weather-fetching module
    data = get_weather(location)

    # ---- Current Weather ----
    current = data.current_condition[0]
    current_info = {
        "tempC": float(current.temp_C),
        "FeelsLikeC": float(current.FeelsLikeC),
        "humidity": int(current.humidity),
        "uvIndex": int(current.uvIndex),
        "windSpeedKmph": int(current.windspeedKmph),
        "pressure": int(current.pressure),
        "desc": current.weatherDesc[0].value,
    }

    # ---- Forecast Weather ----
    daily_records = []
    for day in data.weather[:forecast_days]:
        # ✅ Retrieve midday weather description (index 4 ≈ 12:00 p.m.)
        midday_desc = (
            day.hourly[4].weatherDesc[0].value if day.hourly else "N/A"
        )

        record = {
            "date": day.date,
            "maxTempC": float(day.maxtempC),
            "minTempC": float(day.mintempC),
            "avgTempC": float(day.avgtempC),
            "uvIndex": int(day.uvIndex),
            "sunHour": float(day.sunHour),
            "weatherDesc": midday_desc,
        }
        daily_records.append(record)

    forecast_df = pd.DataFrame(daily_records)

    # ✅ Return combined structure (current + forecast)
    return {
        "current_condition": current_info,
        "forecast": forecast_df
    }

In [4]:
# Heplers for the unified the unit. 
def _to_float(x, default=pd.NA):
    try:
        return float(x)
    except (TypeError, ValueError):
        return default

def _val_or(default, *path):
        try:
            obj = path[0]
            for p in path[1:]:
                obj = getattr(obj, p)
            return obj if obj not in (None, "") else default
        except Exception:
            return default
        
def _first_value(lst, default=""):
        # lists like [WeatherDesc(value='Sunny')]
        try:
            return getattr(lst[0], "value")
        except Exception:
            return default
        
def _uv_level(uvi: float) -> str:
        if uvi is None:
            return "N/A"
        if uvi < 3:  return "Low"
        if uvi < 6:  return "Moderate"
        if uvi < 8:  return "High"
        if uvi < 11: return "Very High"
        return "Extreme"
    
def _beaufort(kmph: float) -> str:
        if kmph is None:
            return "N/A"
        # thresholds ~ Beaufort (km/h)
        bft = [
            (1, "0 Calm"), (6, "1 Light Air"), (12, "2 Light Breeze"),
            (20, "3 Gentle Breeze"), (29, "4 Moderate Breeze"),
            (39, "5 Fresh Breeze"), (50, "6 Strong Breeze"),
            (62, "7 Near Gale"), (75, "8 Gale"), (89, "9 Severe Gale"),
            (103, "10 Storm"), (118, "11 Violent Storm")
        ]
        for th, label in bft:
            if kmph <= th:
                return label
        return "12 Hurricane"
    
def _dew_point_c(t_c: float, rh: float):
        # Magnus approximation; returns None if inputs missing
        t = _to_float(t_c, None); h = _to_float(rh, None)
        if t is None or h is None:
            return None
        # constants for water over 0–60°C
        a, b = 17.62, 243.12
        import math
        try:
            gamma = (a * t) / (b + t) + math.log(max(1e-6, h/100.0))
            return round((b * gamma) / (a - gamma), 1)
        except Exception:
            return None

In [5]:
# Detailed current weather inforamtion

def get_current_weather(city: str, grouped: bool = False) -> dict:
    """
    Fetch and return a display-ready dict of current weather for `city`.
    If grouped=True, return a sectioned dict for card-based rendering.
    """
    raw = get_weather(city)
    cc = raw.current_condition[0]
    area = raw.nearest_area[0]
    today = raw.weather[0] if getattr(raw, "weather", None) else None
    astro = today.astronomy[0] if (today and getattr(today, "astronomy", None)) else None

    # ---------- wind formatting ----------
    wind_kmph = _val_or(None, cc, "windspeedKmph")
    wind_dir  = _val_or(None, cc, "winddir16Point")
    wind_deg  = _val_or(None, cc, "winddirDegree")
    if wind_kmph and wind_dir and wind_deg:
        wind = f"{wind_kmph} km/h {wind_dir} ({wind_deg}°) • {_beaufort(_to_float(wind_kmph, None))}"
    elif wind_kmph and wind_dir:
        wind = f"{wind_kmph} km/h {wind_dir} • {_beaufot(_to_float(wind_kmph, None))}"
    elif wind_kmph:
        wind = f"{wind_kmph} km/h • {_beaufort(_to_float(wind_kmph, None))}"
    else:
        wind = "N/A"

    # ---------- computed extras ----------
    temp_c   = _to_float(_val_or(None, cc, "temp_C"), None)
    feels_c  = _to_float(_val_or(None, cc, "FeelsLikeC"), None)
    rh       = _to_float(_val_or(None, cc, "humidity"), None)
    uvi      = _to_float(_val_or(None, cc, "uvIndex"), None)

    feels_delta = None if (temp_c is None or feels_c is None) else round(feels_c - temp_c, 1)
    dew_c       = _dew_point_c(temp_c, rh)  # may be None

    # ---------- flat dict (display-ready with units) ----------
    data = {
        "City": city,
        "Date & Time": _val_or("N/A", cc, "localObsDateTime"),

        "Suburb":  _first_value(_val_or([], area, "areaName"), ""),
        "State":   _first_value(_val_or([], area, "region"), "N/A"),
        "Country": _first_value(_val_or([], area, "country"), "N/A"),
        "Latitude":  _val_or("N/A", area, "latitude"),
        "Longitude": _val_or("N/A", area, "longitude"),

        "Temperature (°C)": None if temp_c is None  else f"{temp_c:.1f} °C",
        "Feels Like (°C)":  None if feels_c is None else f"{feels_c:.1f} °C",
        "Feels Delta (°C)": None if feels_delta is None else f"{feels_delta:+.1f}",
        "Weather Description": _first_value(_val_or([], cc, "weatherDesc"), ""),
        "Weather Code": _val_or("N/A", cc, "weatherCode"),

        "Humidity (%)":   None if rh is None else f"{rh:.0f}%",
        "Dew Point (°C)": None if dew_c is None else f"{dew_c:.1f} °C",
        "Pressure (hPa)": None if _to_float(_val_or(None, cc, "pressure"), None) is None
                           else f"{_to_float(getattr(cc, 'pressure', None)):.0f}",
        "Cloud Cover (%)": None if _to_float(_val_or(None, cc, "cloudcover"), None) is None
                           else f"{_to_float(getattr(cc, 'cloudcover', None)):.0f}%",
        "Visibility (km)": None if _to_float(_val_or(None, cc, "visibility"), None) is None
                           else f"{_to_float(getattr(cc, 'visibility', None)):.0f} km",

        "Wind": wind,
        "Precip (mm)": None if _to_float(_val_or(None, cc, "precipMM"), None) is None
                        else f"{_to_float(getattr(cc, 'precipMM', None)):.1f} mm",

        "UV Index": "N/A" if uvi is None else f"{uvi:.0f} ({_uv_level(uvi)})",
        "Sunrise":  _val_or("N/A", astro, "sunrise") if astro else "N/A",
        "Sunset":   _val_or("N/A", astro, "sunset")  if astro else "N/A",
    }

    # Clean up the NA
    data = {k: v for k, v in data.items() if v is not None}

    if not grouped:
        return data

In [6]:
# ---- ☁️ Brief Current Weather ----
def brief_current_weather(city: str):
    """
    Retrieve and summarise the current weather for a given city.

    This function fetches weather data via `get_weather(city)` and
    extracts key information (Temperature, Feels Like, Condition, UV Index)
    into a one-row pandas DataFrame for quick display.

    Args:
        city (str): Name of the city to retrieve weather for.

    Returns:
        pandas.DataFrame: One-row DataFrame containing:
            - Temperature (°C)
            - Feels Like (°C)
            - Condition (weather description)
            - UV Index
    """

    raw = get_weather(city)

    # 🩵 If `get_weather()` returns a string, attempt to parse as JSON
    if isinstance(raw, str):
        import json
        try:
            raw = json.loads(raw)
        except Exception as e:
            raise ValueError(f"Invalid response format: {e}")

    # 🩵 If response is a dictionary (typical for API JSON)
    if isinstance(raw, dict):
        # Handle common structure: {'data': {...}} or direct 'current_condition'
        if "data" in raw:
            raw = raw["data"]
        if "current_condition" in raw:
            cc = raw["current_condition"][0]
        else:
            raise ValueError("Missing 'current_condition' key in weather data.")
    else:
        # Fallback: support legacy object-based responses
        cc = raw.current_condition[0]

    # ---- Extract UV Index ----
    uv = None
    if hasattr(cc, "uvIndex"):
        uv = float(cc.uvIndex)
    elif hasattr(cc, "UVIndex"):
        uv = float(cc.UVIndex)
    elif isinstance(cc, dict) and "uvIndex" in cc:
        uv = float(cc["uvIndex"])

    # ---- Build display DataFrame ----
    df = pd.DataFrame({
        "Temperature (°C)": [
            float(cc["temp_C"]) if isinstance(cc, dict) else float(cc.temp_C)
        ],
        "Feels Like (°C)": [
            float(cc["FeelsLikeC"]) if isinstance(cc, dict) else float(cc.FeelsLikeC)
        ],
        "Condition": [
            cc["weatherDesc"][0]["value"] if isinstance(cc, dict)
            else cc.weatherDesc[0].value
        ],
        "UV Index": [uv]
    })

    return df

In [7]:
# ---- ⏰ Hourly Weather Data Extraction ----
def hourly_weather_data(weather_data):
    """
    Extract hourly weather information from a DailyForecast object.

    This function processes a list of DailyForecast data (typically returned
    by the API or the `get_weather()` function) and retrieves hourly values
    for temperature, feels-like temperature, humidity, and UV index.

    Args:
        weather_data (list[DailyForecast]): A list containing daily forecast objects.

    Returns:
        pandas.DataFrame: A DataFrame containing hourly records with columns:
            - time (str): Time in HH:00 format (e.g., "06:00")
            - tempC (float): Actual temperature in Celsius
            - FeelsLikeC (float): Feels-like temperature in Celsius
            - humidity (int): Relative humidity percentage
            - uvIndex (int): UV index value
    """

    hourly_records = []

    # ✅ Extract hourly weather data (default: today's forecast)
    for hour in weather_data[0].hourly:
        record = {
            "time": f"{int(hour.time)//100:02d}:00",   # Convert 600 → "06:00"
            "tempC": float(hour.tempC),
            "FeelsLikeC": float(hour.FeelsLikeC),
            "humidity": int(hour.humidity),
            "uvIndex": int(hour.uvIndex)
        }
        hourly_records.append(record)

    # ✅ Convert to DataFrame and preserve time order
    df = pd.DataFrame(hourly_records)
    df["time"] = pd.Categorical(df["time"], categories=df["time"], ordered=True)

    return df

## 📊 Visualisation Functions

In [8]:
# Visualise the detailed current data into a better presentation

def group_weather_data(data: dict) -> dict:
    """
    Group a flat weather data dict (from get_current_weather)
    into logical sections for card-style visualisation.

    Parameters
    ----------
    data : dict
        The flat dictionary returned by get_current_weather().

    Returns
    -------
    dict
        A grouped dictionary, where keys are section titles and
        values are sub-dicts containing label-value pairs.
    """

    sections = {
        "Now": {
            "Temperature (°C)": data.get("Temperature (°C)"),
            "Feels Like (°C)":  data.get("Feels Like (°C)"),
            "Feels Delta (°C)": data.get("Feels Delta (°C)"),
            "Weather Description": data.get("Weather Description"),
            "Weather Code": data.get("Weather Code"),
        },
        "Air": {
            "Humidity (%)": data.get("Humidity (%)"),
            "Dew Point (°C)": data.get("Dew Point (°C)"),
            "Pressure (hPa)": data.get("Pressure (hPa)"),
            "Cloud Cover (%)": data.get("Cloud Cover (%)"),
            "Visibility (km)": data.get("Visibility (km)"),
            "Precip (mm)": data.get("Precip (mm)"),
        },
        "Wind": {
            "Wind": data.get("Wind"),
        },
        "Sun & UV": {
            "UV Index": data.get("UV Index"),
            "Sunrise":  data.get("Sunrise"),
            "Sunset":   data.get("Sunset"),
        },
        "Location": {
            "City": data.get("City"),
            "Suburb": data.get("Suburb"),
            "State": data.get("State"),
            "Country": data.get("Country"),
            "Latitude": data.get("Latitude"),
            "Longitude": data.get("Longitude"),
            "Date & Time": data.get("Date & Time"),
        },
    }

    # Clean up the NA
    def _clean(d: dict) -> dict:
        return {k: v for k, v in d.items() if v not in (None, "N/A", "")}

    sections = {title: _clean(items) for title, items in sections.items() if _clean(items)}
    return sections

In [9]:
# ---- 🎨 Render Detailed Weather Cards ----
def _render_detail_cards(sections: dict, framework: str = "streamlit", **kwargs):
    """
    Render grouped weather data in a soft purple 'glassmorphism' theme using HTML and CSS.

    This function creates a responsive two-column card layout displaying
    weather information categories (e.g., Now, Air, Wind, Sun & UV, Location).
    Each card is styled with gradient backgrounds, soft shadows, and icons.

    Args:
        sections (dict): Dictionary of grouped weather data, e.g.:
            {
                "Now": {"Temperature": "22°C", "Humidity": "60%"},
                "Wind": {"Speed": "15 km/h", "Direction": "NE"}
            }
        framework (str): Optional rendering context ("streamlit" or "notebook").
        **kwargs: Additional arguments for future flexibility.

    Returns:
        None: Displays formatted HTML content directly in Jupyter Notebook or Streamlit.
    """
    from IPython.display import HTML, display

    # 💜 Section colors and icons
    section_colors = {
        "Now": "#f3e9ff",
        "Air": "#efe6ff",
        "Wind": "#f2e8ff",
        "Sun & UV": "#f7f0ff",
        "Location": "#f1eaff"
    }
    title_icons = {
        "Now": "🌤️",
        "Air": "🌫️",
        "Wind": "🌬️",
        "Sun & UV": "☀️",
        "Location": "📍"
    }

    # 💅 CSS styling: two-column grid layout with gradient glass background
    style_html = """
    <style>
    .detail-wrapper {
        background: linear-gradient(180deg, #f7f3ff 0%, #ffffff 90%);
        border-radius: 20px;
        padding: 30px 35px 40px 35px;
        width: 96%;
        max-width: 1300px;
        margin: 25px auto;
        box-shadow: 0 6px 14px rgba(140, 100, 255, 0.08);
        font-family: 'Segoe UI','Helvetica Neue',sans-serif;
    }

    /* 🌸 Responsive two-column layout */
    .detail-grid {
        display: grid;
        grid-template-columns: repeat(auto-fill, minmax(500px, 1fr));
        gap: 25px;
        justify-items: center;
    }

    .detail-card {
        background-color: rgba(255, 255, 255, 0.8);
        backdrop-filter: blur(6px);
        border-radius: 18px;
        box-shadow: 0 3px 10px rgba(130, 90, 255, 0.12);
        padding: 20px 28px;
        width: 100%;
        max-width: 520px;
        color: #3a2c5f;
        transition: transform 0.25s ease, box-shadow 0.25s ease;
    }
    .detail-card:hover {
        transform: scale(1.02);
        box-shadow: 0 6px 14px rgba(120, 80, 220, 0.25);
    }

    .detail-card-title {
        font-size: 18px;
        font-weight: 650;
        color: #5b3cc4;
        margin-bottom: 10px;
        background-color: rgba(155,130,255,0.1);
        padding: 6px 12px;
        border-radius: 8px;
        display: inline-block;
        box-shadow: inset 0 1px 2px rgba(255,255,255,0.5);
    }

    .detail-row {
        display: flex;
        justify-content: space-between;
        margin: 5px 0;
        font-size: 15px;
    }
    .detail-row span:first-child {
        font-weight: 500;
        color: #45307e;
    }
    .detail-row span:last-child {
        color: #2e1f55;
        font-weight: 550;
    }
    </style>
    """

    # 💜 Render individual section cards
    cards_html = ""
    for title, kv in sections.items():
        if not kv:
            continue
        bg = section_colors.get(title, "#f3e8ff")
        icon = title_icons.get(title, "")
        rows = "".join(
            f"<div class='detail-row'><span>{k}</span><span>{v}</span></div>"
            for k, v in kv.items()
        )
        cards_html += f"""
        <div class="detail-card" style="background:{bg};">
            <div class="detail-card-title">{icon} {title}</div>
            {rows}
        </div>
        """

    # 💜 Combine all cards into a single HTML structure
    html = f"""
    {style_html}
    <div class="detail-wrapper">
        <div class="detail-grid">
            {cards_html}
        </div>
    </div>
    """

    # ✅ Display HTML content
    display(HTML(html))

In [10]:
# ---- 📊 Hourly Weather Data Visualisation ----
def visualisation_hourly_weather(df):
    """
    Display interactive hourly weather charts with a purple-themed UI.

    This function creates an interactive Plotly line chart to visualise
    hourly weather data such as temperature, humidity, feels-like temperature,
    and UV index. Users can switch between metrics using vertical toggle buttons.

    Args:
        df (pandas.DataFrame): DataFrame containing hourly weather data with columns:
            - time (str)
            - tempC (float)
            - FeelsLikeC (float)
            - humidity (int)
            - uvIndex (int)

    Returns:
        None: Displays interactive chart and controls directly in Jupyter Notebook.
    """

    import plotly.express as px
    import ipywidgets as widgets
    from IPython.display import display, clear_output

    # --- Output container for rendering the chart ---
    plot_output = widgets.Output()

    # --- Internal plotting function ---
    def draw_chart(metric):
        with plot_output:
            clear_output(wait=True)

            # 🎨 Color and title mapping for each metric
            color_map = {
                "tempC": "#7b5ce1",
                "FeelsLikeC": "#c57aff",
                "humidity": "#3cc47b",
                "uvIndex": "#f6b93b"
            }
            title_map = {
                "tempC": "🌡️ Hourly Temperature (°C)",
                "FeelsLikeC": "🌀 Hourly Feels Like (°C)",
                "humidity": "💧 Hourly Humidity (%)",
                "uvIndex": "☀️ Hourly UV Index"
            }

            # --- Create interactive Plotly line chart ---
            fig = px.line(
                df,
                x="time",
                y=metric,
                markers=True,
                title=title_map[metric],
                color_discrete_sequence=[color_map[metric]],
                hover_data={"time": True, metric: ':.1f'}
            )

            # --- Style lines and markers ---
            fig.update_traces(
                line=dict(width=3),
                marker=dict(size=8, line=dict(width=1.5, color='white'))
            )

            # --- Configure layout (purple theme aesthetic) ---
            fig.update_layout(
                template="plotly_white",
                plot_bgcolor="#f9f7ff",
                paper_bgcolor="white",
                title_font=dict(size=18, color="#5b3cc4", family="Segoe UI"),
                font=dict(color="#4b3ca8", family="Segoe UI"),
                hoverlabel=dict(bgcolor="#f2eaff", font_size=13, font_family="Segoe UI"),
                margin=dict(l=50, r=30, t=60, b=40),
            )
            fig.update_xaxes(title="Time", showgrid=True, gridcolor="#e0d7ff")
            fig.update_yaxes(title=metric, showgrid=True, gridcolor="#e0d7ff")

            # ✅ Force chart rendering
            fig.show()

    # --- Create right-side metric toggle panel (vertical layout) ---
    toggle = widgets.ToggleButtons(
        options=[
            ("🌡️ Temperature", "tempC"),
            ("🌀 Feels Like", "FeelsLikeC"),
            ("💧 Humidity", "humidity"),
            ("☀️ UV Index", "uvIndex")
        ],
        value="tempC",
        layout=widgets.Layout(
            flex_flow='column',
            align_items='stretch',
            width='220px'
        ),
        style={'font_weight': 'bold'}
    )

    # --- Inject CSS styling for purple 3D toggle design ---
    style_html = widgets.HTML("""
    <style>
    .jupyter-widgets.widget-toggle-buttons {
        background-color: transparent !important;
        display: flex !important;
        flex-direction: column !important;
        gap: 10px !important;
    }
    .jupyter-widgets.widget-toggle-buttons .widget-toggle-button {
        background-color: #ede2ff !important;
        color: #4b0082 !important;
        border: 2px solid #cbb3ff !important;
        border-radius: 12px !important;
        font-weight: 600 !important;
        box-shadow: 0 3px 8px rgba(130, 100, 230, 0.15) !important;
        transition: all 0.25s ease-in-out !important;
        padding: 6px 10px !important;
        text-align: left !important;
    }
    .jupyter-widgets.widget-toggle-buttons .widget-toggle-button:hover {
        background-color: #c9b3ff !important;
        color: white !important;
        transform: translateY(-2px);
    }
    .jupyter-widgets.widget-toggle-buttons .widget-toggle-button.mod-active {
        background-color: #8462ff !important;
        color: white !important;
        border-color: #a68aff !important;
        box-shadow: 0 3px 8px rgba(90, 60, 200, 0.3) !important;
    }
    </style>
    """)

    # --- Toggle button behaviour ---
    def on_change(change):
        if change["name"] == "value" and change["new"] != change["old"]:
            draw_chart(change["new"])

    toggle.observe(on_change)

    # --- Layout: Left (plot) + Right (metric selector) ---
    metric_card = widgets.VBox([
        widgets.HTML("<div style='font-weight:700;color:#5b3cc4;margin-bottom:8px;'>📊 Select Metric</div>"),
        toggle
    ])
    visual_box = widgets.HBox([
        plot_output,
        widgets.VBox([style_html, metric_card],
                     layout=widgets.Layout(margin='0 0 0 25px'))  # Right margin spacing
    ])

    # --- Initial display (default: Temperature chart) ---
    draw_chart("tempC")
    display(visual_box)

In [11]:
# ---- 🌈 Forecast Weather Visualisation ----
def show_forecast_weather(location):
    """
    Display a 3-day weather forecast layout with responsive card design and Plotly temperature chart.

    This function fetches the forecast data for a specified location and presents it
    in a balanced, full-width layout that visually matches the current weather display.
    It features animated loading, gradient styling, and a temperature trend graph.

    Args:
        location (str): Name of the city or region to display forecast for.

    Returns:
        None: Displays styled HTML content and Plotly visualisations directly
              in the Jupyter Notebook output cell.
    """

    import plotly.express as px
    import time
    from IPython.display import display, HTML, clear_output

    # ---- 🌀 Loading animation ----
    display(HTML("""
    <div id='loading-forecast' style='text-align:center; margin-top:40px; font-family:Segoe UI; color:#5b3cc4;'>
        <div style='display:inline-block; width:22px; height:22px;
                    border:3px solid #d8c6ff; border-top:3px solid #5b3cc4;
                    border-radius:50%; animation:spin 1s linear infinite;
                    margin-right:8px; vertical-align:middle;'></div>
        <b>Loading forecast data...</b>
        <style>@keyframes spin {0%{transform:rotate(0deg);}100%{transform:rotate(360deg);}}</style>
    </div>
    """))
    time.sleep(1.0)

    try:
        # ✅ Fetch processed forecast data from API
        weather_data = get_weather_data(location)
        df_forecast = weather_data["forecast"]

        # Clear the loading message
        with brief_output:
            clear_output(wait=True)

        # ---- 💅 Responsive layout style ----
        style_html = """
        <style>
        .forecast-title {
            text-align: center;
            font-weight: 700;
            font-size: 20px;
            color: #5b3cc4;
            margin: 20px 0 10px 0;
            text-shadow: 0 1px 3px rgba(140,100,255,0.25);
        }

        .forecast-container {
            display: flex;
            flex-wrap: wrap;
            justify-content: center;
            background-color: #f7f3ff;
            border-radius: 18px;
            padding: 18px 25px;
            width: 95%;
            margin: 15px auto 25px auto;
            box-shadow: 0 4px 12px rgba(140, 100, 255, 0.1);
            font-family: 'Segoe UI', 'Helvetica Neue', sans-serif;
            max-width: 1300px;
        }

        .forecast-card {
            background-color: white;
            border-radius: 15px;
            padding: 16px 20px;
            margin: 10px;
            text-align: center;
            width: 220px;
            box-shadow: 0 4px 10px rgba(120, 80, 220, 0.12);
            transition: transform 0.25s ease, box-shadow 0.25s ease;
        }
        .forecast-card:hover {
            transform: scale(1.04);
            box-shadow: 0 6px 14px rgba(90, 60, 200, 0.25);
        }

        .forecast-date {
            font-weight: 650;
            font-size: 16px;
            color: #5b3cc4;
            margin-bottom: 5px;
        }
        .forecast-temp {
            font-size: 15px;
            font-weight: 600;
            margin: 4px 0;
            color: #3a2c6b;
        }
        .forecast-desc {
            font-style: italic;
            color: #6b5b95;
            margin-top: 6px;
            font-size: 14px;
        }
        </style>
        """

        # ---- 🧩 Generate forecast cards for each day ----
        card_html = "".join([
            f"""
            <div class="forecast-card">
                <div class="forecast-date">{row.date}</div>
                <div class="forecast-temp">🌡️ {row.maxTempC}°C / {row.minTempC}°C</div>
                <div>☀️ UV: {row.uvIndex} | 🌞 {row.sunHour}h</div>
                <div class="forecast-desc">{row.weatherDesc}</div>
            </div>
            """ for _, row in df_forecast.iterrows()
        ])

        # ---- 📋 Combine title and forecast cards ----
        display(HTML(style_html + f"""
        <div class="forecast-title">📅 3-Day Weather Forecast — {location}</div>
        <div class="forecast-container">{card_html}</div>
        """))

        # ---- 📈 Temperature trend line chart (Plotly) ----
        df_plot = df_forecast.melt(
            id_vars=["date"],
            value_vars=["maxTempC", "minTempC"],
            var_name="Type",
            value_name="Temperature (°C)"
        )
        df_plot["Type"] = df_plot["Type"].replace({
            "maxTempC": "Max Temp (°C)",
            "minTempC": "Min Temp (°C)"
        })

        fig = px.line(
            df_plot,
            x="date",
            y="Temperature (°C)",
            color="Type",
            markers=True,
            color_discrete_map={
                "Max Temp (°C)": "#7b5ce1",
                "Min Temp (°C)": "#ffb347"
            },
            hover_name="Type",
            hover_data={"date": True, "Temperature (°C)": ':.1f'},
            title=f"🌤️ Temperature Trend — {location}"
        )

        # ---- 🎨 Styling the Plotly chart ----
        fig.update_traces(
            line=dict(width=2.8),
            marker=dict(size=7, line=dict(width=1.3, color='white'))
        )

        fig.update_layout(
            template="plotly_white",
            plot_bgcolor="#f9f7ff",
            paper_bgcolor="white",
            title_font=dict(size=18, color="#5b3cc4", family="Segoe UI"),
            font=dict(color="#4b3ca8", family="Segoe UI"),
            hoverlabel=dict(bgcolor="#f2eaff", font_size=12, font_family="Segoe UI"),
            legend=dict(title=None, orientation="h", y=1.1, x=0.25),
            margin=dict(l=40, r=30, t=70, b=40),
            width=1000,
            height=440
        )
        fig.update_xaxes(showgrid=True, gridwidth=0.5, gridcolor="#e0d7ff", tickangle=-25)
        fig.update_yaxes(showgrid=True, gridwidth=0.5, gridcolor="#e0d7ff")

        # ✅ Display final chart
        display(fig)

    except Exception as e:
        clear_output(wait=True)
        display(HTML(f"<span style='color:red;'>❌ Unable to load forecast data: {e}</span>"))

## 🤖 Natural Language Processing

In [12]:
# ---- 💬 NLP Parsing Functions ----

# Predefined list of recognised cities for entity matching
KNOWN_CITIES = [
    "Perth, Australia", "Sydney, Australia", "Melbourne, Australia", "Brisbane, Australia",
    "Adelaide, Australia", "Canberra, Australia", "Hobart, Australia", "Darwin, Australia",
    "Auckland, New Zealand", "Wellington, New Zealand", "Christchurch, New Zealand",
    "Singapore, Singapore", "Seoul, South Korea", "San Francisco, USA", "Seattle, USA",
    "Sapporo, Japan", "Stockholm, Sweden", "Santiago, Chile", "São Paulo, Brazil",
    "Shanghai, China", "Shenzhen, China", "Shenyang, China", "Suzhou, China", "Nanchang, China",
    "Ganzhou, China"
]


def parse_city(text: str) -> str | None:
    """
    Extract a city name from the user's text input.

    The function scans the predefined KNOWN_CITIES list and matches either the
    full city name (e.g., "Sydney, Australia") or the standalone city name
    (e.g., "Sydney"). Matching is case-insensitive.

    Args:
        text (str): User input text (any language supported).

    Returns:
        str | None: The matched city name (e.g., "Sydney, Australia") or None if no match is found.
    """
    t = text.lower()

    # --- Direct full-name match (e.g., "Sydney, Australia") ---
    for c in KNOWN_CITIES:
        if c.lower() in t:
            return c

    # --- Fallback: match by city keyword only (e.g., "Sydney") ---
    for c in KNOWN_CITIES:
        name = c.split(",")[0].strip().lower()
        if re.search(rf"\b{re.escape(name)}\b", t):
            return c

    # --- No valid match found ---
    return None


def parse_need(text: str) -> str:
    """
    Identify what type of weather information the user is requesting.

    This function detects keywords in the user's text and classifies the
    request into one of the following categories: 'uv', 'humidity', 'wind',
    'pressure', 'brief', or 'detail'.

    Args:
        text (str): User input text.

    Returns:
        str: A label representing the detected request category.
             Possible values:
             - 'uv'       → UV index related query
             - 'humidity' → Humidity related query
             - 'wind'     → Wind speed or direction query
             - 'pressure' → Air pressure query
             - 'detail'   → Detailed forecast request
             - 'brief'    → General weather summary (default)
    """
    t = text.lower()

    # --- Detect UV index requests ---
    if "uv" in t or "ultraviolet" in t:
        return "uv"

    # --- Detect humidity requests ---
    if "humidity" in t:
        return "humidity"

    # --- Detect wind-related requests ---
    if "wind" in t:
        return "wind"

    # --- Detect pressure-related requests ---
    if "pressure" in t or "barometer" in t:
        return "pressure"

    # --- Detect detailed forecast requests ---
    if "detail" in t or "more" in t:
        return "detail"

    # --- Default: general or brief summary ---
    return "brief"

## 🧭 User Interface

In [13]:
# ---- 💜 Global Soft Purple Theme (supports ipywidgets 8+) ----
from IPython.display import HTML, display

style_html = """
<style>
/* 💜 Global soft purple theme — compatible with ipywidgets 8+ */
body, .widget-label, .widget-html-content {
    font-family: 'Segoe UI', 'Helvetica Neue', sans-serif !important;
}

/* ---- Top Section ---- */
.top-section {
    background-color: #f3e8ff;
    border-radius: 15px;
    padding: 15px 20px;
    box-shadow: 0 3px 10px rgba(130, 90, 255, 0.15);
    margin-bottom: 20px;
}
.title {
    color: #5b3cc4;
    font-size: 22px;
    font-weight: 650;
    margin-bottom: 10px;
}

/* ---- Button Styles (Current / Forecast Weather) ---- */
.widget-button {
    background-color: #e9dbff !important;
    color: #4b0082 !important;
    border-radius: 10px !important;
    border: 2px solid #c3aaff !important;
    font-weight: 600 !important;
    transition: all 0.25s ease !important;
}
.widget-button:hover {
    background-color: #c9b3ff !important;
    color: white !important;
    transform: translateY(-2px);
}
.widget-button.mod-active {
    background-color: #8462ff !important;
    color: white !important;
}

/* ---- Toggle Buttons (vertical column style) ---- */
.widget-toggle-buttons,
.widget-toggle-buttons > div[role="group"] {
    background-color: transparent !important;  /* ✅ Remove default background box */
    border-radius: 0 !important;               /* ✅ Remove rounded corners */
    padding: 0 !important;                     /* ✅ Remove inner padding */
    display: flex !important;
    flex-direction: column !important;
    gap: 10px !important;
}
.widget-toggle-buttons button {
    background-color: #ede2ff !important;
    color: #4b0082 !important;
    border: 2px solid #cbb3ff !important;
    border-radius: 10px !important;
    font-weight: 600 !important;
    box-shadow: 0 2px 6px rgba(100, 70, 200, 0.15) !important;
    transition: all 0.25s ease !important;
}
.widget-toggle-buttons button:hover {
    background-color: #c9b3ff !important;
    color: white !important;
}
.widget-toggle-buttons button.mod-active {
    background-color: #8462ff !important;
    color: white !important;
    border-color: #a68aff !important;
}

/* ---- Tab Styling (Description / Visualisation) ---- */
.widget-tab > .p-TabBar-tab {
    background-color: #f1e8ff !important;
    color: #4b0082 !important;
    border-radius: 8px !important;
    font-weight: 600 !important;
    margin-right: 4px !important;
    transition: all 0.25s ease-in-out !important;
}
.widget-tab > .p-TabBar-tab:hover {
    background-color: #c9b3ff !important;
    color: white !important;
}
.widget-tab > .p-TabBar-tab.p-mod-current {
    background-color: #8462ff !important;
    color: white !important;
    box-shadow: 0 2px 6px rgba(90, 60, 200, 0.25) !important;
}

/* ---- Metric Card Title ---- */
.metric-card h4 {
    color: #5b3cc4;
    font-weight: 700;
    font-size: 16px;
    text-align: center;
    margin-bottom: 12px;
}
</style>
"""
display(HTML(style_html))

In [14]:
# ---- 🌤️ Top Section ----
title = widgets.HTML("<div class='title'>🌤️ Weather Advisor — Control Panel</div>")
subtitle = widgets.HTML("<div class='subtitle'>Select a function below to explore weather data or chat with our assistant.</div>")

# ---- 🌸 Two main action buttons ----
btn_current = widgets.Button(
    description='Current Weather',
    layout=widgets.Layout(width='150px', height='35px'),
    style={'button_color': '#c7a4ff', 'font_weight': 'bold'}
)
btn_forecast = widgets.Button(
    description='Forecast Weather',
    layout=widgets.Layout(width='150px', height='35px'),
    style={'button_color': '#c7a4ff', 'font_weight': 'bold'}
)

# ---- 🏙️ City input area ----
city_input = widgets.Text(
    placeholder='Enter a city name (e.g. Sydney, Australia)',
    description='City:',
    style={'description_width': '60px'},
    layout=widgets.Layout(width='300px')
)
confirm_btn = widgets.Button(
    description='Confirm',
    layout=widgets.Layout(width='100px', height='35px'),
    style={'button_color': '#b388ff', 'font_weight': 'bold'}
)
city_section = widgets.HBox([city_input, confirm_btn])
city_container = widgets.VBox([], layout=widgets.Layout(margin='20px 0 0 0'))

# ---- Output sections ----
output = widgets.Output()
brief_output = widgets.Output()
details_output = widgets.Output()

# ---- Control logic ----
selected_action = widgets.Text(value='', layout=widgets.Layout(display='none'))

# ✅ Button click logic for switching between Current and Forecast modes
def show_city_input(btn):
    """
    Enhanced button handler supporting seamless transition
    between Current and Forecast Weather modules.
    """
    import time
    city_container.children = [city_section]
    selected_action.value = btn.description

    # ✅ Reset all outputs
    with output:
        clear_output(wait=True)
        display(HTML(f"<b style='color:#5b3cc4;'>You selected:</b> {btn.description}"))
        display(brief_output, details_output)

    with brief_output:
        clear_output(wait=True)
    with details_output:
        clear_output(wait=True)

    city = city_input.value.strip()
    if city:
        time.sleep(0.3)
        if btn.description == "Forecast Weather":
            with brief_output:
                clear_output(wait=True)  # ✅ Clear current-weather content completely
                show_forecast_weather(city)
        elif btn.description == "Current Weather":
            with brief_output:
                clear_output(wait=True)
            on_confirm_click(None)     

btn_current.on_click(show_city_input)
btn_forecast.on_click(show_city_input)


# ---- Confirm button logic ----
def on_confirm_click(_):
    """
    Improved version: supports unlimited city queries
    without losing output binding.
    """
    import time
    city = city_input.value.strip()
    action = selected_action.value

    if not city:
        with output:
            clear_output(wait=True)
            display(HTML("<span style='color:red;'>⚠️ Please enter a city name first.</span>"))
        return

    # Step 1. Clear previous output but retain widget binding
    with brief_output:
        clear_output(wait=True)
    with details_output:
        clear_output(wait=True)
    with output:
        clear_output(wait=True)
        display(brief_output, details_output)

    # Step 2. Loading animation
    with brief_output:
        display(HTML(f"""
        <div style='text-align:center; margin-top:20px; font-family:Segoe UI; color:#5b3cc4;'>
            <div style='display:inline-block; width:22px; height:22px;
                        border:3px solid #d8c6ff; border-top:3px solid #5b3cc4;
                        border-radius:50%; animation:spin 1s linear infinite;
                        margin-right:8px; vertical-align:middle;'></div>
            <b>Loading weather data for <span style="color:#7b5ce1;">{city}</span>...</b>
            <style>@keyframes spin {{0% {{transform: rotate(0deg);}} 100% {{transform: rotate(360deg);}}}}</style>
        </div>
        """))
    time.sleep(1.0)

    # Step 3. Load and display weather data
    with brief_output:
        clear_output(wait=True)
        try:
            if action == "Current Weather":
                df = brief_current_weather(city)

                # Header card
                display(HTML(f"""
                <div style='background:#f8f3ff; border-radius:15px;
                    padding:18px 25px; margin-top:10px; box-shadow:0 3px 10px rgba(140,100,255,0.1);'>
                    <div style='font-weight:bold; color:#5b3cc4; font-size:18px; margin-bottom:10px;'>
                        📍 {city}
                    </div>
                """))

                # Data output
                for col, val in df.iloc[0].items():
                    display(HTML(f"<div style='margin:4px 0;'><b>{col}:</b> {val}</div>"))
                display(HTML("</div><br>"))
                display(show_details_btn)

            elif action == "Forecast Weather":
                show_forecast_weather(city)

            else:
                display(HTML("<i style='color:gray;'>⚙️ Unknown action.</i>"))

        except Exception as e:
            display(HTML(f"<span style='color:red;'>❌ Unable to fetch data: {e}</span>"))


confirm_btn.on_click(on_confirm_click)
city_input.on_submit(on_confirm_click)


# ---- 🟣 Detailed Weather Section ----
show_details_btn = widgets.Button(
    description='Show Detailed Weather Information',
    layout=widgets.Layout(width='300px', height='35px', margin='10px 0 0 0'),
    style={'button_color': '#d1b3ff', 'font_weight': 'bold'}
)


def show_detailed_weather(_):
    """
    Display the detailed weather interface with both Description
    and Visualisation tabs, styled in unified purple theme.
    """
    with details_output:
        clear_output(wait=True)
        city = city_input.value.strip()
        if not city:
            display(HTML("<span style='color:red;'>⚠️ Please enter a city name first.</span>"))
            return

        # Loading animation
        display(HTML(f"""
        <div style='text-align:center; margin-top:20px; font-family:Segoe UI; color:#5b3cc4;'>
          <div style='display:inline-block; width:25px; height:25px;
                      border:3px solid #d8c6ff; border-top:3px solid #5b3cc4;
                      border-radius:50%; animation:spin 1s linear infinite; margin-right:10px;'>
          </div>
          <b>Loading detailed weather for <span style='color:#7b5ce1'>{city}</span>...</b>
          <style>
          @keyframes spin {{ 0% {{ transform: rotate(0deg); }} 100% {{ transform: rotate(360deg); }} }}
          </style>
        </div>
        """))
        time.sleep(1.2)

        try:
            # Fetch current weather data
            data = get_current_weather(city)
            grouped = group_weather_data(data)

            # Hide brief output
            brief_output.layout.display = 'none'

            # Title header
            title_html = widgets.HTML(f"""
            <div style='text-align:center;font-weight:700;font-size:22px;
                        color:#5b3cc4;font-family:"Segoe UI", Helvetica, sans-serif;
                        margin:10px 0 15px 0;text-shadow:0 1px 3px rgba(150,100,255,0.2);'>
                🌈 Detailed Weather for <span style="color:#5b3cc4;">{city}</span>
            </div>
            """)

            # Tabs for Description / Visualisation
            tab = widgets.ToggleButtons(
                options=[('📄 Description', 'desc'), ('📊 Visualisation', 'vis')],
                value='desc',
                layout=widgets.Layout(justify_content='center', width='auto', margin='8px 15px 0'),
                style={'font_weight': 'bold'}
            )

            # ---- Unified purple theme styling ----
            display(widgets.HTML("""
            <style>
            .jupyter-widgets.widget-toggle-buttons {
                display:flex; justify-content:center; gap:10px;
            }
            .jupyter-widgets.widget-toggle-buttons .widget-toggle-button {
                background-color:#f7f1ff;
                border:2px solid #d8c6ff;
                border-radius:10px;
                font-family:'Segoe UI', sans-serif;
                font-size:15px;
                font-weight:600;
                color:#5b3cc4;
                padding:8px 22px;
                box-shadow:0 2px 6px rgba(140,100,255,0.15);
                transition:all 0.25s ease-in-out;
            }
            .jupyter-widgets.widget-toggle-buttons .widget-toggle-button:hover {
                background-color:#e8d9ff;
                transform:translateY(-2px);
            }
            .jupyter-widgets.widget-toggle-buttons .widget-toggle-button.mod-active {
                background-color:#8d6bff !important;
                color:white !important;
                border-color:#8d6bff !important;
                box-shadow:0 4px 10px rgba(130,90,255,0.35);
            }
            </style>
            """))

            # Description Tab
            desc_output = widgets.Output()
            with desc_output:
                display(HTML("<div class='detail-section'>"))
                _render_detail_cards(grouped)
                display(HTML("</div>"))

            # Visualisation Tab
            vis_output = widgets.Output()
            with vis_output:
                try:
                    raw_data = get_weather(city)
                    if hasattr(raw_data, "weather"):
                        weather_data = raw_data.weather
                    elif isinstance(raw_data, dict):
                        possible_keys = [k for k in raw_data.keys() if "weather" in k or "hour" in k]
                        if possible_keys:
                            weather_data = raw_data[possible_keys[0]]
                        else:
                            raise KeyError("No valid 'weather' field found.")
                    else:
                        raise TypeError(f"Unexpected data type: {type(raw_data)}")

                    df_today = hourly_weather_data(weather_data)
                    display(HTML(f"<p style='color:#5b3cc4;'>✅ Loaded {len(df_today)} hourly records for {city}.</p>"))
                    visualisation_hourly_weather(df_today)

                except Exception as e:
                    display(HTML(f"<div style='color:red;'>⚠️ Error generating visualisation: {e}</div>"))

            # Toggle behaviour
            content = widgets.VBox([desc_output])
            def switch(change):
                if change['name'] == 'value':
                    content.children = [desc_output if change['new'] == 'desc' else vis_output]
            tab.observe(switch, names='value')

            # Final display
            details_output.layout.display = 'block'
            with details_output:
                clear_output(wait=True)
                display(widgets.VBox([title_html, tab, content]))

        except Exception as e:
            details_output.clear_output()
            display(HTML(f"<span style='color:red;'>❌ Unable to load detailed weather: {e}</span>"))


show_details_btn.on_click(show_detailed_weather)


# ---- 💬 Smart Weather Chatbot ----
current_city = None
current_need = None
chat_history = []

def chatbot_response(msg: str) -> str:
    """
    AI-driven chatbot that provides weather information
    based on user queries and live API data.
    """
    global current_city, current_need, chat_history

    chat_history.append(msg)
    msg_lower = msg.lower()

    # Step 1: Parse city
    city = parse_city(msg)
    if city:
        current_city = city

    # Step 2: Parse user intent
    need = parse_need(msg)
    if need:
        current_need = need

    # Step 3: Handle missing context
    if not current_city:
        return "🌍 Which city are you asking about?"
    if not current_need:
        return f"🤔 What do you want to know about {current_city}? (e.g., UV, humidity, wind, or temperature)"

    # Step 4: Fetch live weather data
    try:
        data = get_weather(current_city)
        cc = data.current_condition[0]
        temp = cc.temp_C
        feels = cc.FeelsLikeC
        humidity = cc.humidity
        pressure = cc.pressure
        uv = cc.uvIndex
        wind = cc.windspeedKmph
        desc = cc.weatherDesc[0].value
    except Exception as e:
        return f"⚠️ Sorry, I couldn’t fetch data for {current_city}. (Error: {e})"

    # Step 5: Respond based on user intent
    if current_need == "uv":
        return f"☀️ In {current_city}, the UV index is {uv}."
    elif current_need == "humidity":
        return f"💧 The humidity in {current_city} is {humidity}%."
    elif current_need == "pressure":
        return f"🌡️ The air pressure in {current_city} is {pressure} hPa."
    elif current_need == "wind":
        return f"🌬️ The wind speed in {current_city} is {wind} km/h."
    elif current_need == "detail":
        return f"📊 {current_city} weather: {desc}, {temp}°C (feels like {feels}°C), humidity {humidity}%, UV {uv}, wind {wind} km/h, pressure {pressure} hPa."
    elif current_need == "brief":
        return f"🌤️ It's {desc.lower()} in {current_city}, around {temp}°C."
    else:
        return f"🤖 In {current_city}: {desc}, {temp}°C, humidity {humidity}%, UV {uv}."


# ---- 💬 Chat UI Elements ----
chat_output = widgets.Output()

chat_input = widgets.Text(
    placeholder='Ask me about the weather...',
    description='You:',
    style={'description_width': '50px'},
    layout=widgets.Layout(width='400px')
)

send_button = widgets.Button(
    description='Send 💜',
    style={'button_color': '#c7a4ff', 'font_weight': 'bold'},
    layout=widgets.Layout(width='100px')
)

def send_message(_):
    """Handle chat message sending and display conversation."""
    text = chat_input.value.strip()
    if not text:
        return
    chat_input.value = ""

    with chat_output:
        display(HTML(f"<b style='color:#4b0082;'>You:</b> {text}"))
        reply = chatbot_response(text)
        display(HTML(f"<b style='color:#5b3cc4;'>WeatherBot:</b> {reply}"))

send_button.on_click(send_message)

chatbox = widgets.VBox([
    widgets.HTML("<div class='chat-title' style='font-weight:bold;color:#5b3cc4;font-size:16px;margin-bottom:5px;'>💬 Chat with WeatherBot</div>"),
    widgets.HBox([chat_input, send_button]),
    chat_output
])


# ---- 🌟 Combine everything into the main UI container ----
def main_app():
    """Combine all sections into one unified UI layout."""
    button_row = widgets.HBox(
        [btn_current, btn_forecast],
        layout=widgets.Layout(justify_content='flex-start', gap='30px')
    )

    top_section = widgets.VBox([title, subtitle, button_row, city_container, output])
    chat_section = chatbox

    app = widgets.VBox([
        top_section,
        chat_section,
        widgets.HTML("<hr style='border:none;height:2px;background:#f3e8ff;margin:15px 0;'>")
    ], layout=widgets.Layout(width='100%', align_items='flex-start'))

    return app

# ✅ Display main application interface
ui = main_app()
display(ui)

C:\Users\angla\AppData\Local\Temp\ipykernel_27984\3525982853.py:149: DeprecationWarning:

on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').

